In [1]:

# ## UNCOMMENT THIS CELL IF USING GOOGLE COLAB ###

# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

# # Enter the path in your Drive..
# FOLDERNAME = "Transformer"

# assert FOLDERNAME is not None, "[!] Enter the foldername."

# # Now that we've mounted your Drive, this ensures that
# # the Python interpreter of the Colab VM can load
# # python files from within it.
# import sys
# sys.path.append(f'/content/drive/My Drive/{FOLDERNAME}')

# path = f'/content/drive/My Drive/{FOLDERNAME}'

Mounted at /content/drive


In [1]:
%load_ext autoreload
%autoreload 2


import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from preprocessor import WMTPreProcessor
from utils import *
from dataset import *
from model import *
from Scheduler import *


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

cuda:0


In [3]:
data_folder = "WMT14 EN-GE/"

vocab_folder = os.path.join(data_folder,"vocab")
train_folder = os.path.join(data_folder,"train")
test_folder = os.path.join(data_folder,"test")
mappings_path = os.path.join(data_folder,"dict.en-de")

folders_map = {"train":train_folder,"test":test_folder}

In [4]:
!pip install wget
import wget

for folder in [data_folder, vocab_folder, train_folder, test_folder]:
  if not(os.path.exists(folder)):
    os.mkdir(folder)

urls_to_load = {"https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en":vocab_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.de":vocab_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en":train_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.en":test_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de":train_folder,
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.de":test_folder}  # a list to store the urls

files_namings = {"https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.en":"vocab.en",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/vocab.50K.de":"vocab.de",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.en":"train.en",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.en":"test.en",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/train.de":"train.de",
                "https://nlp.stanford.edu/projects/nmt/data/wmt14.en-de/newstest2012.de":"test.de"}

# starting to download
print("Starting downloading")
for url,path in urls_to_load.items():
    file_name = path + '/' + files_namings[url] # get the full path to the file
    if not(os.path.exists(file_name)):
     # if doesn't exists, download it.
        file_name = wget.download(url, out=file_name)
        print(file_name)

print("ok")

Starting downloading
ok


In [5]:
wmt_preprocessor = WMTPreProcessor(vocabPath = vocab_folder, mappings_path = mappings_path)
wmt_preprocessor.initialize()

Building Vocab ....:   0%|          | 0/50001 [00:00<?, ?it/s]

Building Vocab ....:   0%|          | 0/50001 [00:00<?, ?it/s]

In [6]:
data_text = {"en":{"train":[],"test":[],"val":[]},
             "de":{"train":[],"test":[],"val":[]}}

data_text = extractTextFromFolders(folders_map,data_text,val_split = 0.75, limit = 5e4)


extracting text from folders ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
data_tokens = extractTokens(data_text,wmt_preprocessor)

extracting tokens ...:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizing ...:   0%|          | 0/37500 [00:00<?, ?it/s]

setting max sent len ....:   0%|          | 0/37500 [00:00<?, ?it/s]

tokenizing ...:   0%|          | 0/3004 [00:00<?, ?it/s]

setting max sent len ....:   0%|          | 0/3004 [00:00<?, ?it/s]

...

setting max sent len ....:   0%|          | 0/37500 [00:00<?, ?it/s]

tokenizing ...:   0%|          | 0/3004 [00:00<?, ?it/s]

setting max sent len ....:   0%|          | 0/3004 [00:00<?, ?it/s]

tokenizing ...:   0%|          | 0/12500 [00:00<?, ?it/s]

setting max sent len ....:   0%|          | 0/12500 [00:00<?, ?it/s]

In [8]:
data_encodings = extractEncodings(data_tokens,wmt_preprocessor)

extracting encodings ...:   0%|          | 0/2 [00:00<?, ?it/s]

encoding ....:   0%|          | 0/37500 [00:00<?, ?it/s]

padding ...:   0%|          | 0/37500 [00:00<?, ?it/s]

encoding ....:   0%|          | 0/3004 [00:00<?, ?it/s]

padding ...:   0%|          | 0/3004 [00:00<?, ?it/s]

...

padding ...:   0%|          | 0/37500 [00:00<?, ?it/s]

encoding ....:   0%|          | 0/3004 [00:00<?, ?it/s]

padding ...:   0%|          | 0/3004 [00:00<?, ?it/s]

encoding ....:   0%|          | 0/12500 [00:00<?, ?it/s]

padding ...:   0%|          | 0/12500 [00:00<?, ?it/s]

In [9]:
wmt_train = WMT(inpt_encodings=data_encodings["en"]["train"],tgt_encodings=data_encodings["de"]["train"], pad_index = wmt_preprocessor.PAD)
wmt_val = WMT(inpt_encodings=data_encodings["en"]["val"],tgt_encodings=data_encodings["de"]["val"], pad_index = wmt_preprocessor.PAD)
wmt_test = WMT(inpt_encodings=data_encodings["en"]["test"],tgt_encodings=data_encodings["de"]["test"], pad_index = wmt_preprocessor.PAD)

In [10]:
batch_size = 64
trainLoaders = {}

train_dataloader = DataLoader(wmt_train, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(wmt_val, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(wmt_test, batch_size=batch_size, shuffle=True)


trainLoaders["train"] = train_dataloader
trainLoaders["val"] = val_dataloader


## Generating an example ##
for i,batch in enumerate(iter(val_dataloader)):

    inputs,targets = batch["input"],batch["target"]
    idx = np.random.randint(0,len(inputs))
    if i < 1:
        print(f"inputs batch shape: {inputs['encodings'].size()}")
        print(f"targets batch shape: {targets['decoder_input_encodings'].size()}")
        sample = (inputs["encodings"][idx],targets["decoder_input_encodings"][idx])
        print(f"encodings : {sample[0]}")
        print(f"masks : {inputs['masks'][idx]}")
        print(f"masks : {inputs['masks'].shape}")
        print(f"masks : {targets['masks'][idx]}")
        print(f"masks : {targets['masks'].shape}")
        print(wmt_preprocessor.decode([sample[0].tolist()], unpad = True, idx2word = wmt_preprocessor.idx2word_en),"\n",
             wmt_preprocessor.decode([sample[1].tolist()], unpad = True, idx2word = wmt_preprocessor.idx2word_de))
        break

inputs batch shape: torch.Size([64, 138])
targets batch shape: torch.Size([64, 146])
encodings : tensor([ 1962,  1762,    80,  3244,    15,    14, 24444,    11,    39,  1728,
        27772,     8,     1,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
            2,     2,     2,     2,     2,   

decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

[['…', 'anyone', 'who', 'believes', 'that', 'is', 'clinging', 'to', 'an', 'ancient', 'cliché', '.', '<EOS>']] 
 [['<START>', '...', 'wer', 'das', 'glaubt', ',', 'lebt', 'mit', 'einem', 'uralten', 'klischee', '.']]


In [11]:
model = Transformer(src_vocabSize = len(wmt_preprocessor.vocab_en),
                    tgt_vocabSize = len(wmt_preprocessor.vocab_de),
                    d_embed = 512,
                    d_model = 512,
                    d_ff = 2056,
                    dropout = 0.1,
                    noEncoder = 1,
                    noDecoder = 1,
                    pad_index = wmt_preprocessor.PAD,
                   device = device).to(device)

In [12]:
optimizer = torch.optim.Adam(model.parameters(),
                             betas = (0.9, 0.98),
                              eps = 1.0e-9)

In [13]:
scheduler = Scheduler(optimizer =  optimizer,
                 dim_embed= 512,
                 warmup_steps=200)

In [14]:
print(f"Numeber of the model's trainable paramaters : {count_parameters(model)}")


Numeber of the model's trainable paramaters : 74887490


In [15]:
## An example of the untrained model prediction ##

out = model(wmt_train[:1]["input"], wmt_train[:1]["target"])

print(out.shape)
print(f"True label : {wmt_preprocessor.decode(wmt_train[:1]['target']['target_encodings'].tolist(), unpad = True, idx2word = wmt_preprocessor.idx2word_de)}")
print(f"Prediction : {wmt_preprocessor.decode(torch.argmax(out,2).tolist(), unpad = True, idx2word = wmt_preprocessor.idx2word_de)}")

## The prediction loss ##
loss = translationLoss(output=out, target=wmt_train[:1]["target"]["target_encodings"].to(device), pad_index = wmt_preprocessor.PAD, label_smoothing = 0.2)
print(loss)

torch.Size([1, 146, 45362])


decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

True label : [['iron', '<unk>', 'ist', 'eine', 'gebrauchs', '##at##-##at##', 'fertige', 'paste', ',', 'die', 'mit', 'einem', '<unk>', 'oder', 'den', 'fingern', 'als', '<unk>', 'in', 'die', '<unk>', '(', 'winkel', ')', 'der', '<unk>', '<unk>', 'aufgetragen', 'wird', '.', '<EOS>']]


decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

Prediction : [['oppositionelle', 'tant', 'anpassbar', 'betrugsfälle', 'schwingt', 'täuscht', 'instructor', 'markets', 'frustration', 'vielfach', 'abschrift', 'transport', 'wot', 'trost', 'ungeachtet', 'aufgestellten', 'hingucker', 'aufenthaltsgenehmigung', 'unklaren', 'victoria', 'menschenrechtsklauseln', '2800', '7.0', 'industrialisierung', 'dat', 'hauptstraßen', 'teguise', 'benachrichtigen', 'vertreibungen', 'fachkräften', 'kurzsichtigen', 'stattfand', 'gemäß', 'leonard', 'verbinden', 'vorgesehenen', 'uran', 'lebensfähig', 'bluff', 'middle', 'maßgebliche', 'koji', 'reims', 'mitgliedstaats', 'arbeitnehmerrechten', 'müdigkeit', 'müdigkeit', 'strukturen', 'begrüßenswerte', 'sekundären', 'strukturprobleme', 'evaluieren', 'image', '19th', 'cacib', 'geehrten', 'relationship', '104', 'proyecto', 'dünn', 'reziprozität', 'ausscheiden', 'muttersprache', 'buchhalter', 'ausgebildet', 'kulturrevolution', 'sdss', 'jazz', 'technischer', 'stadtviertel', 'markt', '1895', 'tac', 'reziprozität', '¾', '

In [16]:
if not (os.path.exists("trained")):
        os.mkdir("trained")

filename = "transformer-weights"

In [ ]:
trainedModel = train_model(model, trainLoaders, translationLoss, optimizer,
                           pad_index = wmt_preprocessor.PAD, label_smoothing = 0.2,
                           scheduler = scheduler, num_epochs=10, device = device, isSave=True,
                           filename = filename, verbose = False)

Epoch 0/9
----------


Predicting ...:   0%|          | 0/586 [00:00<?, ?it/s]

train prev epoch Loss: 0
train current epoch Loss: 15.056157527116378, lr = 0.0018240887669305594


Predicting ...:   0%|          | 0/196 [00:00<?, ?it/s]

val prev epoch Loss: 0
val current epoch Loss: 14.615887821937093, lr = 0.0018240887669305594
Epoch 1/9
----------


Predicting ...:   0%|          | 0/586 [00:00<?, ?it/s]

train prev epoch Loss: 15.056157527116378
train current epoch Loss: 13.941221852351374, lr = 0.0012903752172114594


Predicting ...:   0%|          | 0/196 [00:00<?, ?it/s]

val prev epoch Loss: 14.615887821937093
val current epoch Loss: 14.321061372756958, lr = 0.0012903752172114594
Epoch 2/9
----------


Predicting ...:   0%|          | 0/586 [00:00<?, ?it/s]

In [ ]:
results = evaluate_model(model, train_dataloader, wmt_preprocessor, device = device)

In [ ]:
model.load_state_dict(torch.load(os.path.join("trained",filename)))

<All keys matched successfully>

In [31]:
for i,batch in enumerate(iter(train_dataloader)):

    inputs,targets = batch["input"],batch["target"]
    idx = np.random.randint(0,len(inputs))
    if i < 1:
        print(f"inputs batch shape: {inputs['encodings'].size()}")
        print(f"targets batch shape: {targets['decoder_input_encodings'].size()}")
        sample = (inputs["encodings"][0],targets["decoder_input_encodings"][0])

        print(f"Input English Sentence : {wmt_preprocessor.decode([sample[0].tolist()], unpad = True, idx2word = wmt_preprocessor.idx2word_en)}")
        print(f"Input German Sentence (Translated) : {wmt_preprocessor.decode([sample[1].tolist()], unpad = True, idx2word = wmt_preprocessor.idx2word_de)}")

        out = model(inputs, targets)
        print(out.shape)

        print(f"Predicted German Sentence (Translated) : {wmt_preprocessor.decode(torch.argmax(out[idx].unsqueeze(0),2).tolist(), unpad = True, idx2word = wmt_preprocessor.idx2word_de)}")

        loss = translationLoss(output = out, target = targets['target_encodings'].to(device), pad_index = wmt_preprocessor.PAD, label_smoothing = 0.2)
        print(loss)

        bleu_score = score(out, targets, wmt_preprocessor, kind = "bleu")
        print(bleu_score)

        break



inputs batch shape: torch.Size([64, 138])
targets batch shape: torch.Size([64, 146])


decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

Input English Sentence : [['<unk>', 'music', 'production', 'is', 'a', 'label', 'for', 'good', 'music', 'with', 'a', 'flavour', 'of', 'jazz', '.', '<EOS>']]


decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

Input German Sentence (Translated) : [['<START>', '<unk>', 'music', 'production', 'ist', 'ein', 'label', 'für', 'gute', 'musik', 'mit', 'einem', 'hauch', 'von', 'jazz', '.']]
torch.Size([64, 146, 45362])


decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

Predicted German Sentence (Translated) : [['das', 'der', 'costa', 'del', '<unk>', 'frontera', '(', 'sie', 'in', 'restaurants', ',', '<unk>', 'von', '<unk>', 'rompido', 'de', 'la', 'maria', '.', '<EOS>']]
tensor(6.0161, device='cuda:0', grad_fn=<AddBackward0>)


decoding ...:   0%|          | 0/64 [00:00<?, ?it/s]

decoding ...:   0%|          | 0/64 [00:00<?, ?it/s]

0.06561609252926305


In [38]:
!cp -r './trained/transformer-weights' /content/drive/MyDrive/

In [39]:
" ".join(['further', ',', 'skype', 'reserves', 'the', 'right', 'to', 'use', 'email', 'or', 'the', 'skype', 'software', 'to', 'notify', 'you', 'of', 'any', 'eventual', 'claims', 'related', 'to', 'your', 'use', 'of', 'skype', 'software', ',', 'the', 'skype', 'websites', 'and', '/', 'or', 'skype', 'products', ',', 'including', 'without', 'limitation', 'claims', 'of', 'violation', 'of', 'third', 'party', 'rights', '.', '<EOS>'])

'further , skype reserves the right to use email or the skype software to notify you of any eventual claims related to your use of skype software , the skype websites and / or skype products , including without limitation claims of violation of third party rights . <EOS>'

In [38]:
infer(model.to(device), "no", wmt_preprocessor, eos_idx = wmt_preprocessor.EOS, device = device)

tokenizing ...:   0%|          | 0/1 [00:00<?, ?it/s]

encoding ....:   0%|          | 0/1 [00:00<?, ?it/s]

padding ...:   0%|          | 0/1 [00:00<?, ?it/s]

decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

Input English Sentence : [['no', '<EOS>']]
input_masks : torch.Size([1, 1, 138])
input_masks : tensor([[[ True,  True, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, F

decoding ...:   0%|          | 0/1 [00:00<?, ?it/s]

[['<START>',
  'können',
  'sie',
  'sich',
  'die',
  '<unk>',
  ',',
  'die',
  '<unk>',
  ',',
  'die',
  '<unk>',
  'und',
  '<unk>',
  'und',
  '<unk>',
  'und',
  '<unk>',
  '.',
  '<EOS>']]